In [ ]:
import pandas as pd
import numpy as np

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']
test_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'click_id']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        

data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train.csv', 
                   #skiprows=range(1,144708152), nrows=181878211-144708152,
                   header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes,
                  engine='c')
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/test.csv', header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train_sample.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)

print('len read:',len(data))


data['hour'] = data["click_time"].dt.hour.astype('uint8')
data['day'] = data["click_time"].dt.day.astype('uint8')

#print('unique days in sample

In [2]:

data1 = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/test.csv', 
                   #skiprows=range(1,144708152), nrows=181878211-144708152,
                   header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes,
                  engine='c')
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/test.csv', header=0,usecols=test_cols,parse_dates=["click_time"], dtype=dtypes)
#data = pd.read_csv('../input/talkingdata-adtracking-fraud-detection/train_sample.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)

data1['hour'] = data1["click_time"].dt.hour.astype('uint8')
data1['day'] = data1["click_time"].dt.day.astype('uint8')
print('len read:',len(data1))

len read: 18790469


In [ ]:
def top_n_values_of_dimensions(data, dimensions,count_col):
    d1 = data[dimensions + [count_col]].\
        groupby(by=dimensions)[[count_col]].count().\
        sort_values(count_col, ascending=False).rename(columns={count_col:'_'.join(dimensions+['count'])})
    d1['_'.join(dimensions+['ratio'])] = d1['_'.join(dimensions+['count'])] / len(data)
    d1 = d1.query('_'.join(dimensions+['ratio']) + ' > 0.01')

        #n_chans = training_hist_attribution[group_by_cols + [counting_col]] \
        #    .groupby(by=group_by_cols)[[counting_col]] \
        #    .count().reset_index().rename(columns={counting_col: feature_name_added1})
    d1=d1.reset_index()
    #d1['_'.join(dimensions)] = pd.Series(['_'. \
    #    join([str(row[dimension]) for dimension in dimensions]) for index, row in d1.iterrows()])
    #data['_'.join(dimensions)] = pd.Series(['_'. \
    #    join([str(row[dimension]) for dimension in dimensions]) for index, row in data.iterrows()])
    
    #data = data.set_index('_'.join(dimensions))
    #data = data.set_index(dimensions)
    
    
    #d1['_'.join(dimensions + ['auc'])] = pd.Series([calculate_auc(data.loc['_'. \
    #    join([str(row[dimension]) for dimension in dimensions])]) for index, row in d1.iterrows()])
    
    #print('dataloc:',data.loc['178_1'])
    
    #print([row for index, row in d1.reset_index().iterrows()])   

    #print(data.loc[[[row[dimension]] for dimension in dimensions] for index, row in d1.reset_index().iterrows()])   
    #print([calculate_auc(data.loc[row[dimensions]]) for index, row in d1.iterrows()])  
    
    #print(data[[dimension for dimension in dimensions]])
    #auc_list = []
    #for index, row in d1.reset_index().iterrows():
    #    query_str = ' & '.join(['%s == \'%d\'' % (dimension, int(row[dimension])) for dimension in dimensions])
        #print('query str:', query_str)
    #    select_data = data.query(query_str)
        #print(select_data)
        #print('auc of select data:', calculate_auc(select_data) )
    #    auc_list.append(calculate_auc(select_data))
    #    l = [str(row[dimension]) for dimension in dimensions]
    #    print("l:%s:%s" % (index,'_'.join(l)))
    #    #print("data loc:", data.loc['_'.join(l)]['is_attributed'].describe())
    #    print("data loc auc: %s" % calculate_auc(data.loc['_'.join(l)]))

    #d1['auc'] = pd.Series(auc_list)

    return d1

import gc
gc.collect()
print('train...')
print(top_n_values_of_dimensions(data, ['channel'],'app'))
print(top_n_values_of_dimensions(data, ['app'],'os'))
print(top_n_values_of_dimensions(data, ['os'],'app'))
print(top_n_values_of_dimensions(data, ['device'],'os'))
print(top_n_values_of_dimensions(data, ['ip'],'os'))
print(top_n_values_of_dimensions(data, ['hour'],'os'))

In [4]:
print('test....')

train...
    channel  channel_count  channel_ratio
0       280       15065927       0.081480
1       245        8873025       0.047987
2       107        8457471       0.045740
3       477        7188340       0.038876
4       134        5924250       0.032040
5       259        5787004       0.031297
6       153        5537977       0.029951
7       265        5446388       0.029455
8       178        5271408       0.028509
9       121        4692438       0.025378
10      205        4359180       0.023575
11      145        3583945       0.019383
12      442        3574611       0.019332
13      459        3477150       0.018805
14      379        3402636       0.018402
15      439        2901762       0.015693
16      489        2711240       0.014663
17      128        2703163       0.014619
18      135        2663913       0.014407
19      466        2655396       0.014361
20      237        2632794       0.014239
21      480        2618441       0.014161
22      469        258838

KeyError: "['is_attributed'] not in index"

In [7]:
def top_n_values_of_dimensions(data, dimensions,count_col):
    d1 = data[dimensions + [count_col]].\
        groupby(by=dimensions)[[count_col]].count().\
        sort_values(count_col, ascending=False).rename(columns={count_col:'_'.join(dimensions+['count'])})
    d1['_'.join(dimensions+['ratio'])] = d1['_'.join(dimensions+['count'])] / len(data)
    #d1 = d1.query('_'.join(dimensions+['ratio']) + ' > 0.01')

        #n_chans = training_hist_attribution[group_by_cols + [counting_col]] \
        #    .groupby(by=group_by_cols)[[counting_col]] \
        #    .count().reset_index().rename(columns={counting_col: feature_name_added1})
    d1=d1.reset_index()
    return d1

#print(top_n_values_of_dimensions(data1, ['channel'],'app'))
#print(top_n_values_of_dimensions(data1, ['app'],'os'))
#print(top_n_values_of_dimensions(data1, ['os'],'app'))
#print(top_n_values_of_dimensions(data1, ['device'],'os'))
#print(top_n_values_of_dimensions(data1, ['ip'],'os'))
#print(top_n_values_of_dimensions(data1, ['hour'],'os'))

train_ch = top_n_values_of_dimensions(data, ['channel'],'app')
test_ch = top_n_values_of_dimensions(data, ['channel'],'app')
mg = train_ch.merge(test_ch, on=['channel'], how='left')


     channel  channel_count_x  channel_ratio_x  channel_count_y  \
0        280         15065927     8.147977e-02         15065927   
1        245          8873025     4.798723e-02          8873025   
2        107          8457471     4.573982e-02          8457471   
3        477          7188340     3.887609e-02          7188340   
4        134          5924250     3.203962e-02          5924250   
5        259          5787004     3.129736e-02          5787004   
6        153          5537977     2.995057e-02          5537977   
7        265          5446388     2.945524e-02          5446388   
8        178          5271408     2.850891e-02          5271408   
9        121          4692438     2.537771e-02          4692438   
10       205          4359180     2.357538e-02          4359180   
11       145          3583945     1.938275e-02          3583945   
12       442          3574611     1.933226e-02          3574611   
13       459          3477150     1.880517e-02          347715

In [38]:
#print(mg.sort_values('channel_ratio_y'))
from pprint import pprint
channel_least_count = 130
print('total ratio of most popular 5 channels in test:')
print(mg.sort_values('channel_ratio_y', ascending=False)['channel_ratio_y'][0:5].sum())
print('total ratio of least popular {} channels in test:'.format(channel_least_count))
print(mg.sort_values('channel_ratio_y')['channel_ratio_y'][0:channel_least_count].sum())

print('most popular 5 channels in test:')
print(','.join(list(mg.sort_values('channel_ratio_y', ascending=False)['channel'][0:5].values.astype(str))))
print('lease popular {} channels in test:'.format(channel_least_count))
pprint(','.join(mg.sort_values('channel_ratio_y')['channel'][0:channel_least_count].values.astype(str)))


total ratio of most popular 5 channels in test:
0.24612252884457975
total ratio of least popular 130 channels in test:
0.05672133236353221
most popular 5 channels in test:
280,245,107,477,134
lease popular 130 channels in test:
'500,146,434,470,172,394,422,354,217,221,227,142,149,165,476,475,473,29,238,162,169,256,251,248,458,471,14,233,225,223,216,181,281,352,407,114,408,222,129,4,498,455,261,272,0,414,311,474,451,465,410,353,15,483,420,341,108,456,332,5,419,486,203,404,488,22,268,174,356,224,479,320,457,126,171,282,262,333,450,274,460,411,138,253,416,322,490,210,160,448,18,453,330,391,24,421,446,484,373,361,208,13,496,182,120,360,277,406,123,30,17,150,449,242,110,118,402,21,467,487,325,417,243,343,124,326,497,113,116,478'


In [ ]:
#print('record counts in days in sample:', data[['day','channel']].groupby(['day']).count())

mean = data[['ip', 'app', 'device', 'os','is_attributed']].groupby(by=['ip', 'app', 'device', 'os'])[['is_attributed']].mean().rename(columns={'is_attributed':'mean'})
count = data[['ip', 'app', 'device', 'os','is_attributed']].groupby(by=['ip', 'app', 'device', 'os'])[['is_attributed']].count().rename(columns={'is_attributed':'count'})

In [ ]:
mean = mean.merge(count, how='left',left_index=True, right_index=True)
print(mean)

In [ ]:
mean.query('count_x > 30').sort_values(by='mean', ascending=False)[0:10]

In [ ]:
joint_col = None
D = 2 ** 26
feature_name_added = 'nc'
training = data
group_by_cols = ['ip','app','device','os']

for col in group_by_cols:
    if joint_col is None:
        joint_col = training[col].astype(str)
    else:
        joint_col = joint_col + "_" + training[col].astype(str)

training['category'] = joint_col.apply(hash) % D
del joint_col

In [ ]:
click_buffer = np.full(D, 3000000000, dtype=np.uint32)
training['epochtime'] = training['click_time'].astype(np.int64) // 10 ** 9
next_clicks = []
for category, time in zip(reversed(training['category'].values),
                          reversed(training['epochtime'].values)):
    next_clicks.append(click_buffer[category] - time)
    click_buffer[category] = time
del (click_buffer)
training[feature_name_added] = list(reversed(next_clicks))

In [ ]:
#print(training[['ip','app','device','os','nc']])

a = mean.query('count_x > 30').sort_values(by='mean', ascending=False)[0:10].reset_index()

In [ ]:
print('done a')
b = a.merge(training[['ip','app','device','os','nc','click_time', 'is_attributed']], 
            on=['ip','app','device','os'], how='left')

In [ ]:
print(b.to_string())

In [ ]:
a1 = mean.query('mean == 0').sort_values(by='count_x', ascending=False)[0:10].reset_index()
print('done a')
b1 = a1.merge(training[['ip','app','device','os','nc','click_time','is_attributed']], on=['ip','app','device','os'], how='left')

In [ ]:
print(b1[8000:10000].to_string())

In [ ]:
print(len(training.query('nc > 1489000000')))
print(len(training))
print(len(mean.query('count_x>30')))
print(len(mean.query('count_x==1')))

In [ ]:
a2 = mean.query('count_x > 30').sort_values(by='mean', ascending=True)[70000:70010].reset_index()
print('done a')
b2 = a2.merge(training[['ip','app','device','os','nc','click_time','is_attributed']], on=['ip','app','device','os'], how='left')

In [ ]:
print(b2[0:1000].to_string())

In [ ]:
#print(data[1000000:1000003])
data['id']=data.index
data1 = data.set_index('click_time').loc['2017-11-08 04:00:00':'2017-11-08 15:00:00']

print(data1['id'].min())
print(data1['id'].max())
print(len(data1))

In [ ]:
data1 = data.set_index('click_time').loc['2017-11-09 04:00:00':'2017-11-09 15:00:00']

print(data1['id'].min())
print(data1['id'].max())
print(len(data1))

In [ ]:
# calculate combined cvr to test hist fts generation

#print('app device cvr:', data[['app','device', 'is_attributed']].groupby(['app','device']).mean().to_string())

os_count = data[['os', 'is_attributed']].groupby(['os']).count()

os_non_attr_count = (1 - data[['os', 'is_attributed']].groupby(['os']).max()) * os_count

print('app device non attri count:', os_non_attr_count.to_string())

#print('app device  attri count:', data[['app','device', 'is_attributed']].groupby(['app','device']).sum().to_string())

#attributed_data = data.query('is_attributed == 1')

#print('len of attributed:',len(attributed_data))

In [ ]:

data= data.merge(os_non_attr_count.reset_index(), on=['os'], how='left')


print(data)

In [ ]:
xxx = pd.DataFrame(data['is_attributed_y'], dtype='float64')
xxx.to_csv('validation.csv.bz2', index=False, compression='bz2')

In [ ]:

log_group = 100000
def rate_calculation(x):
    """Calculate the attributed rate. Scale by confidence"""
    rate = x.sum() / float(x.count())
    conf = np.min([1, np.log(x.count()) / log_group])
    return rate * conf

yyy = data[['app','device', 'is_attributed_x']].groupby(['app','device'])[['is_attributed_x']].apply(rate_calculation).reset_index()
data= data.merge(yyy, on=['app','device'], how='left')

xxx = pd.DataFrame(data['is_attributed_x_y'])
xxx.to_csv('validation1.csv.bz2', index=False, compression='bz2')

In [ ]:
print('sampling data')
#data = data.set_index('ip').loc[lambda x: (x.index + 401) % 10 == 0].reset_index()


data.drop('click_time', inplace=True, axis=1)

In [ ]:
data0['hour'] = data0["click_time"].dt.hour.astype('uint8')
data0['day'] = data0["click_time"].dt.day.astype('uint8')

data0.drop('click_time', inplace=True, axis=1)

In [ ]:
#data.drop('click_time', inplace=True, axis=1)

print('added hour and day')



In [ ]:
#print(data.describe())

data0 = data0.query('hour in [{}]'.format(','.join(map(lambda x: str(x), set(data['hour'].unique())))))
print(data0['hour'].unique())

hour_set = set(data['hour'].unique())
hour_set0 = set(data0['hour'].unique())


In [ ]:
osset = set(data['os'].unique())
osset0 = set(data0['os'].unique())

print(len(osset & osset0))
print(len(osset0))

In [ ]:
chset = set(data['app'].unique())
chset0 = set(data0['app'].unique())

print(len(chset & chset0))
print(len(chset0))